In [1]:
!pip install transformers datasets
!pip install torch

In [2]:
from datasets import load_dataset

train_dataset = load_dataset("csv", data_files='autocast_train_questions_combined_with_tf_negated.csv')

Found cached dataset csv (/home/jupyter/.cache/huggingface/datasets/csv/default-002aee8738d63e56/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
train_dataset = train_dataset['train']
train_dataset = train_dataset.train_test_split(test_size=0.2)

In [4]:
eval_dataset = train_dataset['test']
train_dataset = train_dataset['train']

In [5]:
train_dataset = train_dataset.remove_columns(['publish_time', 'close_time', 'source_links', 'prediction_count', 'forecaster_count', 'crowd', 'question_negated', 'Unnamed: 0', 
                                              'id', 'tags', 'status'])
eval_dataset = eval_dataset.remove_columns(['publish_time', 'close_time', 'source_links', 'prediction_count', 'forecaster_count', 'crowd', 'question_negated', 'Unnamed: 0', 
                                              'id', 'tags', 'status'])

In [6]:
train_set = train_dataset
eval_set = eval_dataset

train_set[0]

{'question': 'Will a human made spaceship enter the Venusian atmosphere before 2030?',
 'background': 'Every space geek loves Mars. Mars gets all the robots as well as all the movies. But whither Venus? Where???s the love for our beautiful goddess neighbor who apparently welcomed the Russian Venera-13 lander by crushing it to death within hours. Informed speculation on Quora suggests that: The [Venera 13] lander\'s seals and pressurized chambers were breached the day of the landing, so all of the material and equipment that the Venera was designed to protect was melted and boiled off decades ago. Yikes. Okay, maybe that\'s why we don\'t drop by more often. Venus is often compared to a literal hellscape. We know about the surface temperatures that melt lead, the sulfuric acid clouds, the poisonous metal snow, etc. But Venus is also interesting! And the clouds of Venus may be the most Earthlike environment in the solar system beyond terra firma. So when will we return to our hot, hostile

In [8]:
filtered_data = [item for item in train_set if item['answer'] is not None or item['choices'] is not None or item['qtype'] != 'num']
filtered_eval = [item for item in eval_set if item['answer'] is not None or item['choices'] is not None or item['qtype'] != 'num']
# print(filtered_data[0])
# print(type(filtered_data))

filtered_data_dict = {idx: item for idx, item in enumerate(filtered_data)}
filtered_eval_dict= {idx: item for idx, item in enumerate(filtered_eval)}

#filtered_data = filtered_data.remove_columns(['qtype'])

In [49]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, TrainingArguments, Trainer
from torch.utils.data import DataLoader

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=12)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
   
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<choices>:"])

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

data = filtered_data_dict
data_eval = filtered_eval_dict

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data

        self.input_ids = []
        self.attention_mask = []
        self.input_texts = []
        
        for idx, i in enumerate(self.data):
            item = self.data[i]
            input_text = f"<startofstring>{item['background']} <question> {item['question']} <choices>: {item['choices']}<endofstring>"
            self.input_texts.append(input_text)
            
        self.inputs = tokenizer(self.input_texts, truncation=False, padding="longest", pad_to_multiple_of = 9)
    
        self.input_ids = self.inputs['input_ids']
        self.attention_mask = self.inputs['attention_mask']
        
        #self.input_ids = self.X_encoded['input_ids']
        #self.attention_mask = self.X_encoded['attention_mask']
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.inputs

train_dataset =  CustomDataset(data,tokenizer)
eval_dataset = CustomDataset(data_eval,tokenizer)

In [30]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.8 MB/s eta 0:00:00a 0:00:01


In [31]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [50]:
"""
class CustomDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        print(batch)
        inputs = [item[0] for item in batch]
        print(inputs)
        targets = [item[1] for item in batch]

        # Merge input tensors
        input_ids = [input["input_ids"] for input in inputs]
        attention_mask = [input["attention_mask"] for input in inputs]
        input_ids = torch.cat(input_ids, dim=0)
        attention_mask = torch.cat(attention_mask, dim=0)

        # Merge target tensors
        targets = torch.stack(targets)

        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": targets}
        
# Create the custom data collator
#data_collator = DataCollatororSequenceClassification()

#train_dataset = CustomDataset(data, tokenizer)
#eval_dataset = CustomDataset(filtered_eval,tokenizer)

# Create the DataLoader
#train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

"""
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from dataclasses import dataclass
import evaluate
from Collator import DataCollatorForSequenceClassification 

model.config.hidden_dropout_prob = 0.5
model.config.attention_probs_dropout_prob = .5

metric = evaluate.load("accuracy")

training_args = TrainingArguments(
    output_dir="output_dir",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=32,
    logging_steps=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy = "epoch")

trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset,
                            eval_dataset=eval_dataset,
                            data_collator=DataCollatorForSequenceClassification(tokenizer))

trainer.train()

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 60364652544 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
trainer.save_model("gpt2-test1")

In [12]:
import torch
from torch.nn.functional import softmax

def predict_answer(model, tokenizer, background, question, choices):
    input_texts = [f"{background} {question} {choice}" for choice in choices]
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)

    # Move input tensors to the device the model is on (GPU or CPU)
    inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Calculate probabilities from logits using softmax
    probabilities = softmax(logits, dim=1)

    # Find the index of the choice with the highest probability
    best_choice_idx = torch.argmax(probabilities, dim=1).item()

    return choices[best_choice_idx]

# Example data
question = data[0]['question']
background = data[0]['background']
choices = data[0]['choices']

tokenizer.pad_token = tokenizer.eos_token

# Get the predicted answer
predicted_answer = predict_answer(model, tokenizer, background, question, choices)
print("Predicted answer:", predicted_answer)

NameError: name 'data' is not defined

In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, TrainingArguments, Trainer
import itertools

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Your custom dataset (replace with your data)
data = filtered_data_dict
    # ...

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        background = item['background']
        question = item['question']
        choices = item['choices']
        answer = item['answer']
        try:
            answer_idx = choices.index(answer)
        except:
            answer_idx = 0

        input_texts = [f"{background} {question} {choice}" for choice in choices]

        # Add max_length=self.max_length to pad or truncate the input sequences
        inputs = self.tokenizer(input_texts, return_tensors="pt", padding='max_length', truncation=True)

        target = torch.tensor(answer_idx, dtype=torch.long)

        return inputs, target

def data_generator(data, tokenizer, batch_size):
    while True:
        # Shuffle the data
        shuffled_data = itertools.permutations(data)
        for batch_data in itertools.zip_longest(*([shuffled_data] * batch_size)):
            # Remove any None values from the batch
            batch_data = [x for x in batch_data if x is not None]
            # Create the batch dataset
            batch_dataset = CustomDataset(batch_data, tokenizer)
            # Yield the batch
            yield batch_dataset[:]

            
class CustomDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        inputs = [item[0] for item in batch]
        targets = [item[1] for item in batch]

        # Merge input tensors
        input_ids = [input["input_ids"] for input in inputs]
        attention_mask = [input["attention_mask"] for input in inputs]
        input_ids = torch.cat(input_ids, dim=0)
        attention_mask = torch.cat(attention_mask, dim=0)

        # Merge target tensors
        targets = torch.stack(targets)

        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": targets}
            
# Create the custom data collator
data_collator = CustomDataCollator(tokenizer)

# Create the DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=data_collator)

training_args = TrainingArguments(
    output_dir="output_dir",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=32,
    logging_steps=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy = "epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset,
    train_dataloader=train_dataloader
)

trainer.train()


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: __init__() got an unexpected keyword argument 'train_dataloader'